In [1]:
from pyspark.sql import SparkSession 
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql.functions import *

/home/pdp28/anaconda3/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
spark = SparkSession.builder.appName('ETL-SupaBase').config("spark.jars","/home/pdp28/spark_jars/postgresql-42.7.5.jar").getOrCreate()

25/04/23 23:01:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark


In [4]:
df = spark.read.format('csv').option('header',True).load('sales_data_sample.csv')
df = df.drop(col('STATUS'), col('ADDRESSLINE1'), col('ADDRESSLINE2'), col('QTR_ID'), col('MONTH_ID'),col('YEAR_ID'))
df.printSchema()
# df.filter(col('POSTALCODE').isNull()).to_pandas_on_spark() #state postalcode 

root
 |-- ORDERNUMBER: string (nullable = true)
 |-- QUANTITYORDERED: string (nullable = true)
 |-- PRICEEACH: string (nullable = true)
 |-- ORDERLINENUMBER: string (nullable = true)
 |-- SALES: string (nullable = true)
 |-- ORDERDATE: string (nullable = true)
 |-- PRODUCTLINE: string (nullable = true)
 |-- MSRP: string (nullable = true)
 |-- PRODUCTCODE: string (nullable = true)
 |-- CUSTOMERNAME: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTALCODE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- TERRITORY: string (nullable = true)
 |-- CONTACTLASTNAME: string (nullable = true)
 |-- CONTACTFIRSTNAME: string (nullable = true)
 |-- DEALSIZE: string (nullable = true)



In [5]:
df_updated = df.withColumn('POSTALCODE' , when(col('POSTALCODE').isNull() , '43230').otherwise(col('POSTALCODE')))


In [6]:
df_updated = df_updated.withColumn('STATE' , when(col('STATE').isNull() , col('COUNTRY')).otherwise(col('STATE')))
df_updated.dropna(how='any')
df_updated = df_updated.withColumn('TERRITORY', when(col('TERRITORY')=='NA', 'North America' ).when(col('TERRITORY')=='APAC', 'Asia‑Pacific')\
                     .when(col('TERRITORY')=='EMEA', 'Europe, Middle East & Africa').otherwise(col('TERRITORY')))

In [7]:
dfs = df_updated.take(4)
dfs = spark.createDataFrame(dfs)
dfs.pandas_api()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.7,2,2871,2/24/2003 0:00,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,NYC,NY,10022,USA,North America,Yu,Kwai,Small
1,10121,34,81.35,5,2765.9,5/7/2003 0:00,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,Reims,France,51100,France,"Europe, Middle East & Africa",Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,Paris,France,75508,France,"Europe, Middle East & Africa",Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.7,8/25/2003 0:00,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,Pasadena,CA,90003,USA,North America,Young,Julie,Medium


In [8]:
schema = '''
OrderNumber STRING,
QuantityOrdered STRING,
PriceEach STRING,
OrderLineNumber STRING,
Sales STRING,
OrderDate STRING,
ProductLine STRING,
Msrp STRING,
ProductCode STRING,
CustomerName STRING,
Phone STRING,
City STRING,
State STRING,
PostalCode STRING,
Country STRING,
Territory STRING,
ContactLastName STRING,
ContactFirstName STRING,
DealSize STRING
'''


In [9]:
df_updated = spark.createDataFrame(df_updated.rdd, schema)
df_updated.pandas_api()

,OrderNumber,QuantityOrdered,PriceEach,OrderLineNumber,Sales,OrderDate,ProductLine,Msrp,ProductCode,CustomerName,Phone,City,State,PostalCode,Country,Territory,ContactLastName,ContactFirstName,DealSize
0,10107,30,95.7,2,2871,2/24/2003 0:00,Motorcycles,95,S10_1678,Land of Toys Inc.,2125557818,NYC,NY,10022,USA,North America,Yu,Kwai,Small
1,10121,34,81.35,5,2765.9,5/7/2003 0:00,Motorcycles,95,S10_1678,Reims Collectables,26.47.1555,Reims,France,51100,France,"Europe, Middle East & Africa",Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Motorcycles,95,S10_1678,Lyon Souveniers,+33 1 46 62 7555,Paris,France,75508,France,"Europe, Middle East & Africa",Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.7,8/25/2003 0:00,Motorcycles,95,S10_1678,Toys4GrownUps.com,6265557265,Pasadena,CA,90003,USA,North America,Young,Julie,Medium
4,10159,49,100,14,5205.27,10/10/2003 0:00,Motorcycles,95,S10_1678,Corporate Gift Ideas Co.,6505551386,San Francisco,CA,43230,USA,North America,Brown,Julie,Medium
5,10168,36,96.66,1,3479.76,10/28/2003 0:00,Motorcycles,95,S10_1678,Technics Stores Inc.,6505556809,Burlingame,CA,94217,USA,North America,Hirano,Juri,Medium
6,10180,29,86.13,9,2497.77,11/11/2003 0:00,Motorcycles,95,S10_1678,Daedalus Designs Imports,20.16.1555,Lille,France,59000,France,"Europe, Middle East & Africa",Rance,Martine,Small
7,10188,48,100,1,5512.32,11/18/2003 0:00,Motorcycles,95,S10_1678,Herkku Gifts,+47 2267 3215,Bergen,Norway,N 5804,Norway,"Europe, Middle East & Africa",Oeztan,Veysel,Medium
8,10201,22,98.57,2,2168.54,12/1/2003 0:00,Motorcycles,95,S10_1678,Mini Wheels Co.,6505555787,San Francisco,CA,43230,USA,North America,Murphy,Julie,Small
9,10211,41,100,14,4708.44,1/15/2004 0:00,Motorcycles,95,S10_1678,Auto Canal Petit,(1) 47.55.6555,Paris,France,75016,France,"Europe, Middle East & Africa",Perrier,Dominique,Medium


In [10]:
df_updated.dropna(how='any').count()

2823

In [11]:
df_updated.write.format('jdbc')\
.option('url','jdbc:postgresql://aws-0-ap-south-1.pooler.supabase.com:5432/postgres')\
.option('user','postgres.nsezfgdhttituosvrecd')\
.option('password','999428@Damo')\
.option('dbtable','Sales_vehicle_csv_tranformed')\
.option('driver','org.postgresql.Driver')\
.mode('overwrite')\
.save()

25/04/23 23:01:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
